In [87]:
from astropy.table import Table, join, hstack, unique
from astropy.io import ascii
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
from glob import glob

import matplotlib as mpl
import matplotlib.pyplot as plt

import os
import requests

## Load SGA Data

In [2]:
SGA = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')

SGA[:10]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32
2,SGA-2020 2,PGC1283207,1283207,228.3770865,5.4232017,S?,152.2,0.36307806,0.724436,0.03463229,23.40448,16.976,False,LEDA-20181114,0,PGC1283207,1,True,228.3770865,5.4232017,0.36307806,2283p055,228.3770803831908,5.423191398593787,0.49470574,SB26,158.20142,0.545691,228.37700918822188,5.4232652570544015,10.897086,3.3509698,3.1147978,3.240862,5.902337,6.9126143,7.941369,8.997992,10.073601,11.199986,12.391357,13.561038,14.841172,16.966799,16.108246,15.486356,16.879545,16.024958,15.400715,16.818878,15.967034,15.341793,16.776297,15.925804,15.300776,16.746685,15.897334,15.272053,16.725166,15.876816,15.2521105,16.708357,15.862035,15.237181,16.696539,15.851936,15.226998,16.689613,15.844313,15.21976,0.013392451,0.02354,0.021872982,0.01736985,0.024445537,0.039866067,0.05026544,0.08455789,0.122911856,0.005682776,0.0054258136,0.0049038026,0.005588406,0.005323561,0.0047632363,0.00543534,0.005177031,0.0046343105,0.0053025587,0.005040888,0.0045181247,0.005206092,0.0049438984,0.0044374703,0.0051483097,0.0048758644,0.0043834248,0.0051032505,0.0048264163,0.004344248,0.0050705094,0.004792021,0.004319857,0.005054293,0.004765629,0.0043044444,16.65942,0.34037337,0.2978292,3.0239506,0.07928849,15.820566,0.2640441,0.34559453,3.3033552,0.003811298,15.195567,0.29826432,0.3001073,3.2333765,0.011723555,0
3,SGA-2020 3,PGC1310416,1310416,202.54443750000002,6.9345944,Sc,159.26,0.4017908,0.7816278,0.073888786,23.498482,16.85,False,LEDA-20181114,1,PGC1310416,1,True,202.54443750000002,6.9345944,0.4017908,

## Match stellar distances

In [3]:
stellar_dist = glob('cache/stellar_dist/*.txt')

In [4]:
stellar_dist[0].split('/')[-1].strip('.txt').strip('EDDtable')

'Virgo_Fornax_SBF'

In [5]:
PGC_ids = []
DM = []
e_DM_hi = []
e_DM_lo = []
dist = []
e_dist = []
catalog = []

for sfile in stellar_dist:
    print(sfile)
    if sfile == 'cache/stellar_dist/EDDtableANGST.txt':
        continue
    name = sfile.split('/')[-1].strip('.txt').strip('EDDtable')
    tab = Table.read(sfile, format = 'csv', header_start = 1, data_start = 5)
    tab.rename_column('pgc', 'PGC')
    jtab = join(SGA, tab, keys = 'PGC', join_type = 'inner')
    print(tab.colnames)
    catalog.append(np.full(np.shape(jtab['PGC'].flatten()), name))
    PGC_ids.append(jtab['PGC'].flatten())
    
    try:
        DM.append(jtab['DM'].flatten())
    except KeyError:
        try:
            DM.append(jtab['dm'].flatten())
        except KeyError:
            try:
                DM.append(jtab['Mod01'].flatten())
            except KeyError:
                try:
                    DM.append(jtab['DM_tip'].flatten())
                except KeyError:
                    try:
                        DM.append(jtab['modulus'].flatten())
                    except KeyError:
                        print('DM not found')
                
    try:
        e_DM_hi.append(jtab['e_dm'].flatten())
    except KeyError:
        try:
            e_DM_hi.append(jtab['edm'].flatten())
        except KeyError:
            try:
                e_DM_hi.append(jtab['e_Mod'].flatten())
            except KeyError:
                try:
                    e_DM_hi.append(jtab['eDM_hi'].flatten())
                except KeyError:
                    try:
                        e_DM_hi.append(jtab['eDM+'].flatten())
                    except KeyError:
                        e_DM_hi.append(np.zeros_like(jtab['modulus'].flatten()))
                        print('E_DM_hi not found')
    
    try:
        e_DM_lo.append(jtab['e_dm'].flatten())
    except KeyError:
        try:
            e_DM_lo.append(jtab['edm'].flatten())
        except KeyError:
            try:
                e_DM_lo.append(jtab['e_Mod'].flatten())
            except KeyError:
                try:
                    e_DM_lo.append(jtab['eDM_lo'].flatten())
                except KeyError:
                    try:
                        e_DM_lo.append(jtab['eDM-'].flatten())
                    except KeyError:
                        e_DM_lo.append(np.zeros_like(jtab['modulus'].flatten()))
                        print('E_DM_lo not found')
                        
#     try:
#         dist.append(jtab['dist'].flatten())
#     except KeyError:
#         try:
#             DM.append(jtab['dm'].flatten())
#         except KeyError:
#             try:
#                 DM.append(jtab['Mod01'].flatten())
#             except KeyError:
#                 try:
#                     DM.append(jtab['DM_tip'].flatten())
#                 except KeyError:
#                     try:
#                         DM.append(jtab['modulus'].flatten())
#                     except KeyError:
#                         print('DM not found')
# #     try:
#         e_DM.append(jtab['e_dm'].flatten())
#     except KeyError:
#         try:
#             e_DM.append(jtab['edm'].flatten())
#         except KeyError:
#             print('e_dm not found')
    # print()
#     dist.append(jtab['dist'].flatten())
#     e_dist.append(jtab['e_d'].flatten())
    
    print('Iteration complete')
    print()
    # print(jtab)
    # break

cache/stellar_dist/EDDtableVirgo_Fornax_SBF.txt
['PGC', '1PGC', 'J2000', 'Name', 'g-z', 'e_gz', 'm_sbf', 'e_msbf', 'DM', 'e_dm', 'dist', 'e_d', 'Altname']
Iteration complete

cache/stellar_dist/EDDtableHST_IR_SBF.txt
['PGC', 'Name', '1PGC', 'Nest', 'DM', 'edm', 'Vhel', 'Ty', 'RA_DEJ', 'glon', 'glat', 'sgl', 'sgb', 'Ag']
Iteration complete

cache/stellar_dist/EDDtableTonry_SBF.txt
['PGC', 'objname', 'Modc', '1PGC', 'Nest', 'al2000', 'de2000', 'V_hel', 'T', 'GrpID', 'AgB', 'VmI', 'e_VmI', 'N_VmI', 'Ibar', 'e_Ibar', 'N_Ibar', 'Mod01', 'e_Mod', 'Rav', 'ratio_in', 'ratio_out', 'Qmax', 'PD', 'NbarI', 'Sigma', 'Qual', 'icnt', 'NBG_ID', 'LH_ID', 'fil']
Iteration complete

cache/stellar_dist/EDDtableMcConnachie.txt
['PGC', 'Name', 'RAJ', 'DeJ', 'E(B-V)', 'DM', 'eDM+', 'eDM-', 'Vhel', 'eV+', 'eV-', 'Vmag', 'emag+', 'emag-', 'PA', 'PA+', 'PA-', 'Ellip', 'eE+', 'eE-', 'SB0', 'eSB+', 'eSB-', 'Re', 'eR+', 'eR-', 'Sig_s', 'eS_s+', 'eS_s-', 'Vrot_s', 'eVr_s+', 'eVr_s-', 'MHI', 'Sig_g', 'eS_g+', 'eS_g-

In [6]:
def flatten_array(arr):
    flat_arr = []
    for i in range(len(arr)):
        for j in range(len(arr[i])):
            flat_arr.append(arr[i][j])
    return flat_arr

In [7]:
PGC_ids_flat = flatten_array(PGC_ids)
DM_flat = flatten_array(DM)
e_DM_lo_flat = flatten_array(e_DM_lo)
e_DM_hi_flat = flatten_array(e_DM_hi)
catalog_flat = flatten_array(catalog)
# for i in range(len(PGC_ids)):
#     for j in range(len(PGC_ids[i])):
#         PGC_ids_flat.append(PGC_ids[i][j])

In [8]:
len(PGC_ids_flat)

520

In [9]:
tab = Table([PGC_ids_flat, DM_flat, e_DM_lo_flat, e_DM_hi_flat, catalog_flat],
             names = ['PGC', 'DM_Stellar', 'e_DM_lo_Stellar', 'e_DM_hi_Stellar', 'Stellar_Catalog'])

In [10]:
tab

PGC,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog
int64,float64,float64,float64,str16
12636,31.532,0.074,0.074,Virgo_Fornax_SBF
12651,31.607,0.065,0.065,Virgo_Fornax_SBF
12691,31.491,0.139,0.139,Virgo_Fornax_SBF
12825,31.483,0.073,0.073,Virgo_Fornax_SBF
12848,31.314,0.075,0.075,Virgo_Fornax_SBF
12878,31.598,0.08,0.08,Virgo_Fornax_SBF
12917,31.47,0.083,0.083,Virgo_Fornax_SBF
12923,31.603,0.068,0.068,Virgo_Fornax_SBF
13028,31.422,0.071,0.071,Virgo_Fornax_SBF


In [11]:
tab['DM_Stellar'].fill_value = -1
tab['e_DM_hi_Stellar'].fill_value = -1
tab['e_DM_lo_Stellar'].fill_value = -1
tab['Stellar_Catalog'].fill_value = -1
tab = tab.filled()
tab

PGC,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog
int64,float64,float64,float64,str16
12636,31.532,0.074,0.074,Virgo_Fornax_SBF
12651,31.607,0.065,0.065,Virgo_Fornax_SBF
12691,31.491,0.139,0.139,Virgo_Fornax_SBF
12825,31.483,0.073,0.073,Virgo_Fornax_SBF
12848,31.314,0.075,0.075,Virgo_Fornax_SBF
12878,31.598,0.08,0.08,Virgo_Fornax_SBF
12917,31.47,0.083,0.083,Virgo_Fornax_SBF
12923,31.603,0.068,0.068,Virgo_Fornax_SBF
13028,31.422,0.071,0.071,Virgo_Fornax_SBF


In [12]:
utab = unique(tab, keys = ['PGC', 'DM_Stellar', 'e_DM_lo_Stellar', 'e_DM_hi_Stellar'])
utab

PGC,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog
int64,float64,float64,float64,str16
218,30.6,0.14,0.14,Tonry_SBF
1014,26.43,0.12,0.12,McConnachi
1037,34.126,0.094,0.094,HST_IR_SBF
1038,26.42,0.04,0.04,McConnachi
1160,31.36,0.33,0.33,Tonry_SBF
1641,26.54,0.04,0.04,McConnachi
2980,31.45,0.47,0.47,Tonry_SBF
3238,26.59,0.06,0.06,McConnachi
3455,34.166,0.081,0.081,HST_IR_SBF


In [13]:
len(SGA)

383620

In [14]:
maintab = join(SGA, utab, keys = 'PGC', join_type = 'left')

In [15]:
maintab[:10]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float64,float64,float64,str16
5000761,SGA-2020 5000761,DR8-3598m132-1566,-1,359.8543401209699,-13.27298450516979,COMP,13.630395,0.36305833,0.37946832,-1.0,24.814651,18.386288,False,DR8,529381,DR8-3598m132-1566,1,True,359.8543401209699,-13.27298450516979,0.36305833,3598m132,359.85434774837023,-13.272977710650537,0.28370878,SB26,168.67378,0.5625282,359.85426448280407,-13.27283520797331,6.5634704,6.3352146,5.8562765,5.2685037,-1.0,0.490355,1.3431504,2.6847167,3.7493393,4.8546333,6.111469,7.3589787,8.511264,-1.0,-1.0,-1.0,23.718433,23.193655,23.154612,21.66221,21.185404,21.111595,20.502134,20.086287,19.971468,20.085892,19.70376,19.57079,19.821558,19.462872,19.34163,19.616253,19.26634,19.164177,19.466738,19.137087,19.014479,19.37565,19.053564,18.928083,-1.0,0.014212543,0.040192965,0.033691984,0.04154174,0.07088843,0.11356775,0.13873318,0.12907872,-1.0,-1.0,-1.0,0.026477985,0.050427496,0.039810665,0.038443595,0.041060727,0.028624054,0.030409802,0.033370476,0.027760057,0.027706875,0.030110046,0.026305534,0.02673246,0.027684188,0.026165267,0.02534611,0.02572179,0.025609946,0.024172347,0.024743635,0.024433767,0.023255061,0.024151104,0.023917114,18.828407,1.8961897,0.27432075,1.2582772,0.5068609,18.54517,1.9884998,0.24010763,1.2152138,0.42738745,18.50474,1.7520914,0.22583209,1.3503227,2.026229,0,--,--,--,--
5002052,SGA-2020 5002052,DR8-0344p257-2109,-1,34.51293949359952,25.744871325366134,DEV,151.64052,0.5850924,0.095845

In [16]:
maintab[maintab['PGC'] == 771919]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float64,float64,float64,str16
868032,SGA-2020 868032,NGC1201,771919,46.03332532673745,-26.06969750695285,E-S0,8.59,3.411929,0.57411647,0.005596872,22.99448,11.701,True,LEDA-20181114,317178,NGC1201,1,True,46.03332532673745,-26.06969750695285,3.411929,0459m260,46.03332172733179,-26.06966853637713,4.9620147,SB26,7.90505,0.56788504,46.033208590750064,-26.069688000976626,118.01962,27.42394,27.187017,27.923996,61.19563,72.84604,83.99373,94.485725,104.98598,115.697334,126.560875,137.64,148.86044,11.54185,10.780679,10.202037,11.4601555,10.698883,10.118637,11.407081,10.646178,10.065561,11.373177,10.6126375,10.031863,11.349672,10.589578,10.0087805,11.333385,10.573862,9.993349,11.321908,10.562697,9.982237,11.313794,10.5551195,9.974818,11.308098,10.5498085,9.96952,0.02019594,0.030068478,0.020640232,0.026782066,0.027618866,0.043218754,0.053795904,0.06738641,0.08724375,0.0021383492,0.0025708186,0.0028877133,0.0019859343,0.0023212722,0.0026519478,0.0018893456,0.002271816,0.002547142,0.0018313518,0.0022027814,0.0024693797,0.0017941092,0.002099232,0.002397022,0.001765516,0.0021255494,0.0023833702,0.0017488543,0.0020479271,0.0023391622,0.0017339756,0.0020891912,0.0023430607,0.0017249141,0.002079005,0.0023316648,11.290278,0.18008748,2974.8523,3.800875,0.0053895037,10.53363,0.17351764,3786.2764,3.9145265,0.014138333,9.953629,0.18070917,3505.5095,3.9080532,0.021632774,0,31.53,0.3,0.3,Tonry_SBF
868032,SGA-2020 868032,NGC1201,771919,

In [17]:
maintab['DM_Stellar'].fill_value = -1
maintab['e_DM_hi_Stellar'].fill_value = -1
maintab['e_DM_lo_Stellar'].fill_value = -1
maintab['Stellar_Catalog'].fill_value = -1

maintab = maintab.filled()

In [18]:
maintab

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float64,float64,float64,str16
5000761,SGA-2020 5000761,DR8-3598m132-1566,-1,359.8543401209699,-13.27298450516979,COMP,13.630395,0.36305833,0.37946832,-1.0,24.814651,18.386288,False,DR8,529381,DR8-3598m132-1566,1,True,359.8543401209699,-13.27298450516979,0.36305833,3598m132,359.85434774837023,-13.272977710650537,0.28370878,SB26,168.67378,0.5625282,359.85426448280407,-13.27283520797331,6.5634704,6.3352146,5.8562765,5.2685037,-1.0,0.490355,1.3431504,2.6847167,3.7493393,4.8546333,6.111469,7.3589787,8.511264,-1.0,-1.0,-1.0,23.718433,23.193655,23.154612,21.66221,21.185404,21.111595,20.502134,20.086287,19.971468,20.085892,19.70376,19.57079,19.821558,19.462872,19.34163,19.616253,19.26634,19.164177,19.466738,19.137087,19.014479,19.37565,19.053564,18.928083,-1.0,0.014212543,0.040192965,0.033691984,0.04154174,0.07088843,0.11356775,0.13873318,0.12907872,-1.0,-1.0,-1.0,0.026477985,0.050427496,0.039810665,0.038443595,0.041060727,0.028624054,0.030409802,0.033370476,0.027760057,0.027706875,0.030110046,0.026305534,0.02673246,0.027684188,0.026165267,0.02534611,0.02572179,0.025609946,0.024172347,0.024743635,0.024433767,0.023255061,0.024151104,0.023917114,18.828407,1.8961897,0.27432075,1.2582772,0.5068609,18.54517,1.9884998,0.24010763,1.2152138,0.42738745,18.50474,1.7520914,0.22583209,1.3503227,2.026229,0,-1.0,-1.0,-1.0,-1
5002052,SGA-2020 5002052,DR8-0344p257-2109,-1,34.51293949359952,25.744871325366134,DEV,151.64052,0.5850924,0.

## SN1a Distances

In [19]:
sn_dist = glob('cache/SN1A_dist/*')
sn_dist[0]

'cache/SN1A_dist/EDDtableConstitution_SNIa.txt'

In [20]:
len(sn_dist)

11

In [21]:
sn_dist[0].split('/')[-1].strip('.txt').strip('EDDtable')

'Constitution_SNI'

In [22]:
PGC_ids = []
DM1 = []
e_DM1 = []
DM2 = []
e_DM2 = []
DM3 = []
e_DM3 = []
DM4 = []
e_DM4 = []
DM5 = []
e_DM5 = []
DM6 = []
e_DM6 = []
DM7 = []
e_DM7 = []
dist = []
e_dist = []
catalog = []

for sfile in sn_dist:
    print(sfile)
    name = sfile.split('/')[-1].strip('.txt').strip('EDDtable')
    tab = Table.read(sfile, format = 'csv', header_start = 1, data_start = 5)
    tab.rename_column('pgc', 'PGC')
    jtab = join(SGA, tab, keys = 'PGC', join_type = 'inner')
    print(tab.colnames)
    catalog.append(np.full(np.shape(jtab['PGC'].flatten()), name))
    PGC_ids.append(jtab['PGC'].flatten())
    
    try:
        ## SALT
        DM1.append(np.array(jtab['S_mu0']).flatten())
        # print(jtab['S_mu0'].flatten())
        e_DM1.append(np.array(jtab['S_emu0']).flatten())
        # print(len(np.array(jtab['S_mu0']).flatten()))
        # print(len(np.array(jtab['S_emu0']).flatten()))
    except KeyError:
        try:
            DM1.append(np.array(jtab['mu0']).flatten())
            e_DM1.append(np.array(jtab['emu0']).flatten())
            # print(len(np.array(jtab['mu0']).flatten()))
            # print(len(np.array(jtab['emu0']).flatten()))
        except KeyError:
            try:
                DM1.append(np.array(jtab['muCV']).flatten())
                e_DM1.append(np.array(jtab['emu']).flatten())
                # print(len(np.array(jtab['muCV']).flatten()))
                # print(len(np.array(jtab['emu']).flatten()))
            except KeyError:
                try:
                    DM1.append(np.array(jtab['mu']).flatten())
                    e_DM1.append(np.array(jtab['emu']).flatten())
                    # print(len(np.array(jtab['mu']).flatten()))
                    # print(len(np.array(jtab['emu']).flatten()))
                except KeyError:
                    try:
                        ## mu0U: Union2 (2010) Distance Modulus assuming H0=74.2 km/s/Mpc
                        DM1.append(np.array(jtab['mu0U']).flatten())
                        e_DM1.append(np.array(jtab['emu0U']).flatten())
                        # print(len(np.array(jtab['mu0U']).flatten()))
                        # print(len(np.array(jtab['emu0U']).flatten()))
                    except KeyError:
                        try:
                            DM1.append(np.array(jtab['DM']).flatten())
                            e_DM1.append(np.array(jtab['eDM']).flatten())
                            # print(len(np.array(jtab['DM']).flatten()))
                            # print(len(np.array(jtab['eDM']).flatten()))
                        except KeyError:
                            try:
                                ## DM5c: Distance modulus, all contributing sources, Cosmicflows-2 zero point
                                DM1.append(np.array(jtab['DM5c']).flatten())
                                e_DM1.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), 0))
                                # print(len(np.array(jtab['DM5c']).flatten()))
                                # print(len(np.array(jtab['eDM5c']).flatten()))
                            except KeyError:
                                print('DM and eDM not found')
    try:
        ## S2_mu0: SALT2
        DM2.append(np.array(jtab['S2_mu0']).flatten())
        e_DM2.append(np.array(jtab['S2_emu0']).flatten())
        # print(len(np.array(jtab['S2_mu0']).flatten()))
        # print(len(np.array(jtab['S2_emu0']).flatten()))
    except KeyError:
        try:
            ## mu0J: Jha et al. (2007) Distance Modulus assuming H0=65 km/s/Mpc 
            DM2.append(np.array(jtab['mu0J']).flatten())
            e_DM2.append(np.array(jtab['emu0J']).flatten())
            # print(len(np.array(jtab['mu0J']).flatten()))
            # print(len(np.array(jtab['emu0J']).flatten()))
        except:
            try:
                ## DM5u: Distance modulus, all contributing sources, Union2 zero point
                DM2.append(np.array(jtab['DM5u']).flatten())
                e_DM2.append(np.array(jtab['DM5u']).flatten())
                # print(len(np.array(jtab['DM5u']).flatten()))
                # print(len(np.array(jtab['DM5u']).flatten()))
            except KeyError:
                DM2.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
                e_DM2.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
    
    try:
        ## M31_mu0
        DM3.append(np.array(jtab['M31_mu0']).flatten())
        e_DM3.append(np.array(jtab['M31_emu0']).flatten())
        # print(len(np.array(jtab['M31_mu0']).flatten()))
        # print(len(np.array(jtab['M31_emu0']).flatten()))
    except KeyError:
        try:
            ## DMu: Distance modulus from Union2 (Amanullah et al. 2010), Union2 zero point 
            DM3.append(np.array(jtab['DMu']).flatten())
            e_DM3.append(np.array(jtab['eDMu']).flatten())
            # print(len(np.array(jtab['DMu']).flatten()))
            # print(len(np.array(jtab['DMu']).flatten()))
        except KeyError:
            DM3.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
            e_DM3.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
    
    try:
        ## M17_mu0
        DM4.append(np.array(jtab['M17_mu0']).flatten())
        e_DM4.append(np.array(jtab['M17_emu0']).flatten())
    except KeyError:
        try:
            ## DMj: Distance modulus from Jha et al. (2007), Union2 zero point
            DM4.append(np.array(jtab['DMj']).flatten())
            e_DM4.append(np.array(jtab['eDMj']).flatten())
        except KeyError:
            DM4.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
            e_DM4.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
            
    try:
        ## DMp: Distance modulus from Prieto et al. (2006), Union2 zero point
        DM5.append(np.array(jtab['DMp']).flatten())
        e_DM5.append(np.array(jtab['eDMp']).flatten())
    except KeyError:
        # print(np.shape(flatten_array(jtab['PGC'])))
        # print(np.shape(np.array(np.array(jtab['PGC'])).flatten()))
        # l = np.array(np.array(jtab['PGC'])).flatten()
        DM5.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
        e_DM5.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
    
    try:
        ## DMh: Distance modulus from Hicken et al. (2009), Union2 zero point
        ## DMf: Distance modulus from Folatelli et al. (2010)
        DM6.append(np.array(jtab['DMh']).flatten())
        DM7.append(np.array(jtab['DMf']).flatten())
        e_DM6.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), 0))
        e_DM7.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), 0))
    except KeyError:
        DM6.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
        DM7.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
        e_DM6.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
        e_DM7.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))

    print()

cache/SN1A_dist/EDDtableConstitution_SNIa.txt
['PGC', 'SNIa', 'zcmb', 'ez', 'S_Bmag', 'S_eB', 'S_s', 'S_es', 'S_c', 'S_ec', 'S_mu0', 'S_emu0', 'S2_Bmag', 'S2_eB', 'S2_s', 'S2_es', 'S2_c', 'S2_ec', 'S2_mu0', 'S2_emu0', 'M31_Del', 'M31_eD', 'M31_AV', 'M31_eAV', 'M31_mu0', 'M31_emu0', 'M17_Del', 'M17_eD', 'M17_AV', 'M17_eAV', 'M17_mu0', 'M17_emu0']

cache/SN1A_dist/EDDtableCSP1_Burns18_SN1a.txt
['PGC', 'SNIa', '1PGC', 'Nest', 'muCV', 'emu', 'zh', 'z3k', 'sBV', 'esBV', 'Dm15', 'eDm15', 'Vmax', 'eVmax', 'EBV', 'eEBV', 'Rv', 'eRv', 'covER', 'Q', 'RA', 'Dec', 'glon', 'glat', 'sgl', 'sgb']

cache/SN1A_dist/EDDtableLOSS_Ganeshalingam13_SN1a.txt
['PGC', 'SNIa', '1PGC', 'Nest', 'RA', 'Dec', 'z3k', 'B', 'eB', 'x1', 'ex1', 'c', 'ec', 'mu', 'emu', 'Sample', 'Ref']

cache/SN1A_dist/EDDtableCSP1_SNIa.txt
['PGC', 'SNIa', 'zhel', 'zcmb', 'E(B-V)', 'DelM15', 'eM15', 'mu0', 'emu0', 'Q']

cache/SN1A_dist/EDDtableSNIa_calibration.txt
['PGC', 'SNIa', 'objname', 'J2000', 'Glon', 'Glat', 'SGL', 'SGB', 'Typ', '

In [23]:
PGC_ids_flat = flatten_array(PGC_ids)
DM1_flat = flatten_array(DM1)
e_DM1_flat = flatten_array(e_DM1)
DM2_flat = flatten_array(DM2)
e_DM2_flat = flatten_array(e_DM2)
DM3_flat = flatten_array(DM3)
e_DM3_flat = flatten_array(e_DM3)
DM4_flat = flatten_array(DM4)
e_DM4_flat = flatten_array(e_DM4)
DM5_flat = flatten_array(DM5)
e_DM5_flat = flatten_array(e_DM5)
DM6_flat = flatten_array(DM6)
e_DM6_flat = flatten_array(e_DM6)
DM7_flat = flatten_array(DM7)
e_DM7_flat = flatten_array(e_DM7)
catalog_flat = flatten_array(catalog)

In [24]:
len(e_DM1)

11

In [25]:
metadata = {
            'DM1_SN': 'SALT data for Constitution_SNIa; Union2 (2010) Distance Modulus assuming H0=74.2 km/s/Mpc for SNIa_calibration; Distance modulus, all contributing sources, Cosmicflows-2 zero point for 5_Sources_SNIa',
            'DM2_SN': 'SALT2 data for Constitution_SNIa; Jha et al. (2007) Distance Modulus assuming H0=65 km/s/Mpc for SNIa_calibration; Distance modulus, all contributing sources, Union2 zero point for 5_Sources_SNIa',
            'DM3_SN': 'MLCS31 data for Constitution_SNIa; Distance modulus from Union2 (Amanullah et al. 2010), Union2 zero point for 5_Sources_SNIa',
            'DM4_SN': 'MLCS17 data for Constitution_SNIa; Distance modulus from Jha et al. (2007), Union2 zero point for 5_Sources_SNIa',
            'DM5_SN': 'Distance modulus from Prieto et al. (2006), Union2 zero point for 5_Sources_SNIa',
            'DM6_SN': 'Distance modulus from Hicken et al. (2009), Union2 zero point for 5_Sources_SNIa (no error given)',
            'DM7_SN': 'Distance modulus from Folatelli et al. (2010) for 5_Sources_SNIa (no error given)'
            }

In [29]:
tab = Table([PGC_ids_flat, DM1_flat, e_DM1_flat, DM2_flat, e_DM2_flat, DM3_flat, e_DM3_flat, DM4_flat, e_DM4_flat,DM5_flat, e_DM5_flat,DM6_flat, e_DM6_flat,DM7_flat, e_DM7_flat, catalog_flat],
             names = ['PGC', 'DM1_SN', 'e_DM1_SN',  'DM2_SN', 'e_DM2_SN', 'DM3_SN', 'e_DM3_SN', 'DM4_SN', 'e_DM4_SN', 'DM5_SN', 'e_DM5_SN', 'DM6_SN', 'e_DM6_SN', 'DM7_SN', 'e_DM7_SN','SN_Catalog'],
             meta = metadata)

In [30]:
tab[:10]

PGC,DM1_SN,e_DM1_SN,DM2_SN,e_DM2_SN,DM3_SN,e_DM3_SN,DM4_SN,e_DM4_SN,DM5_SN,e_DM5_SN,DM6_SN,e_DM6_SN,DM7_SN,e_DM7_SN,SN_Catalog
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,str24
250,35.272,0.185,35.241,0.204,35.282,0.192,35.279,0.177,-1.0,-1.0,-1.0,-1,-1.0,-1,Constitution_SNI
415,35.358,0.185,35.383,0.212,35.248,0.188,35.282,0.187,-1.0,-1.0,-1.0,-1,-1.0,-1,Constitution_SNI
515,0.0,0.0,34.574,0.245,34.037,0.216,34.33,0.212,-1.0,-1.0,-1.0,-1,-1.0,-1,Constitution_SNI
2332,34.962,0.202,34.91,0.227,34.974,0.196,34.858,0.202,-1.0,-1.0,-1.0,-1,-1.0,-1,Constitution_SNI
2501,36.685,0.157,36.769,0.184,36.763,0.17,36.776,0.158,-1.0,-1.0,-1.0,-1,-1.0,-1,Constitution_SNI
2559,35.128,0.193,34.951,0.214,35.032,0.217,35.065,0.192,-1.0,-1.0,-1.0,-1,-1.0,-1,Constitution_SNI
2819,0.0,0.0,0.0,0.0,34.449,0.213,34.464,0.206,-1.0,-1.0,-1.0,-1,-1.0,-1,Constitution_SNI
3508,0.0,0.0,36.51,0.194,36.666,0.158,36.597,0.16,-1.0,-1.0,-1.0,-1,-1.0,-1,Constitution_SNI
3773,0.0,0.0,34.117,0.237,34.17,0.213,34.094,0.213,-1.0,-1.0,-1.0,-1,-1.0,-1,Constitution_SNI


In [33]:
maintab2 = join(maintab, tab, keys = 'PGC', join_type = 'left')
maintab2[:10]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog,DM1_SN,e_DM1_SN,DM2_SN,e_DM2_SN,DM3_SN,e_DM3_SN,DM4_SN,e_DM4_SN,DM5_SN,e_DM5_SN,DM6_SN,e_DM6_SN,DM7_SN,e_DM7_SN,SN_Catalog
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float64,float64,float64,str16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,str24
5000761,SGA-2020 5000761,DR8-3598m132-1566,-1,359.8543401209699,-13.27298450516979,COMP,13.630395,0.36305833,0.37946832,-1.0,24.814651,18.386288,False,DR8,529381,DR8-3598m132-1566,1,True,359.8543401209699,-13.27298450516979,0.36305833,3598m132,359.85434774837023,-13.272977710650537,0.28370878,SB26,168.67378,0.5625282,359.85426448280407,-13.27283520797331,6.5634704,6.3352146,5.8562765,5.2685037,-1.0,0.490355,1.3431504,2.6847167,3.7493393,4.8546333,6.111469,7.3589787,8.511264,-1.0,-1.0,-1.0,23.718433,23.193655,23.154612,21.66221,21.185404,21.111595,20.502134,20.086287,19.971468,20.085892,19.70376,19.57079,19.821558,19.462872,19.34163,19.616253,19.26634,19.164177,19.466738,19.137087,19.014479,19.37565,19.053564,18.928083,-1.0,0.014212543,0.040192965,0.033691984,0.04154174,0.07088843,0.11356775,0.13873318,0.12907872,-1.0,-1.0,-1.0,0.026477985,0.050427496,0.039810665,0.038443595,0.041060727,0.028624054,0.030409802,0.033370476,0.027760057,0.027706875,0.030110046,0.026305534,0.02673246,0.027684188,0.026165267,0.02534611,0.02572179,0.025609946,0.024172347,0.024743635,0.024433767,0.023255061,0.024151104,0.023917114,18.828407,1.8961897,0.27432075,1.2582772,0.

In [34]:
maintab2['DM1_SN'].fill_value = -1
maintab2['e_DM1_SN'].fill_value = -1
maintab2['DM2_SN'].fill_value = -1
maintab2['e_DM2_SN'].fill_value = -1
maintab2['DM3_SN'].fill_value = -1
maintab2['e_DM3_SN'].fill_value = -1
maintab2['DM4_SN'].fill_value = -1
maintab2['e_DM4_SN'].fill_value = -1
maintab2['DM5_SN'].fill_value = -1
maintab2['e_DM5_SN'].fill_value = -1
maintab2['DM6_SN'].fill_value = -1
maintab2['e_DM6_SN'].fill_value = -1
maintab2['DM7_SN'].fill_value = -1
maintab2['e_DM7_SN'].fill_value = -1
maintab2['SN_Catalog'].fill_value = -1

maintab2 = maintab2.filled()

In [63]:
maintab2

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog,DM1_SN,e_DM1_SN,DM2_SN,e_DM2_SN,DM3_SN,e_DM3_SN,DM4_SN,e_DM4_SN,DM5_SN,e_DM5_SN,DM6_SN,e_DM6_SN,DM7_SN,e_DM7_SN,SN_Catalog
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float64,float64,float64,str16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,str24
5000761,SGA-2020 5000761,DR8-3598m132-1566,-1,359.8543401209699,-13.27298450516979,COMP,13.630395,0.36305833,0.37946832,-1.0,24.814651,18.386288,False,DR8,529381,DR8-3598m132-1566,1,True,359.8543401209699,-13.27298450516979,0.36305833,3598m132,359.85434774837023,-13.272977710650537,0.28370878,SB26,168.67378,0.5625282,359.85426448280407,-13.27283520797331,6.5634704,6.3352146,5.8562765,5.2685037,-1.0,0.490355,1.3431504,2.6847167,3.7493393,4.8546333,6.111469,7.3589787,8.511264,-1.0,-1.0,-1.0,23.718433,23.193655,23.154612,21.66221,21.185404,21.111595,20.502134,20.086287,19.971468,20.085892,19.70376,19.57079,19.821558,19.462872,19.34163,19.616253,19.26634,19.164177,19.466738,19.137087,19.014479,19.37565,19.053564,18.928083,-1.0,0.014212543,0.040192965,0.033691984,0.04154174,0.07088843,0.11356775,0.13873318,0.12907872,-1.0,-1.0,-1.0,0.026477985,0.050427496,0.039810665,0.038443595,0.041060727,0.028624054,0.030409802,0.033370476,0.027760057,0.027706875,0.030110046,0.026305534,0.02673246,0.027684188,0.026165267,0.02534611,0.02572179,0.025609946,0.024172347,0.024743635,0.024433767,0.023255061,0.024151104,0.023917114,18.828407,1.8961897,0.27432075,1.2582772,0.

In [39]:
maintab2.write('SGA_distances.fits', format = 'fits')

## Combining with DESI Rosettes

In [48]:
centers = Table.read('cache/tiles-sv3.ecsv', format = 'ascii.ecsv')
centers[0]['TILEID']

1

In [77]:
# m = (maintab2['DM_Stellar'] != -1) or (maintain2['DM1_SN'] != -1)
disttab = maintab2[(maintab2['DM_Stellar'] != -1) | (maintab2['DM1_SN'] != -1)]
disttab

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog,DM1_SN,e_DM1_SN,DM2_SN,e_DM2_SN,DM3_SN,e_DM3_SN,DM4_SN,e_DM4_SN,DM5_SN,e_DM5_SN,DM6_SN,e_DM6_SN,DM7_SN,e_DM7_SN,SN_Catalog
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float64,float64,float64,str16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,str24
207638,SGA-2020 207638,NGC7814,218,0.812559,16.1456194,Sab,134.53,4.365158,0.4305266,0.0035124298,23.421482,11.593,False,LEDA-20181114,73938,NGC7814,1,True,0.812559,16.1456194,4.365158,0009p162,0.8124274854816511,16.1455169162983,11.15105,SB26,134.2024,0.4849028,0.8120762037951955,16.14534837690191,200.04625,82.82556,78.24203,72.30593,94.85372,118.66765,144.39467,172.24834,202.94002,234.82027,267.05215,302.1296,334.5315,11.391324,10.4479065,9.623682,11.228865,10.295919,9.4892435,11.108571,10.186071,9.391858,11.016354,10.10016,9.319667,10.945187,10.035046,9.264916,10.894269,9.989213,9.225079,10.856926,9.954834,9.195862,10.830645,9.929288,9.171985,10.814159,9.9131775,9.156266,0.21526095,0.23602074,0.34232575,0.3225715,0.26907054,0.4221382,0.53733534,0.39212316,0.53470826,0.0020625296,0.0022948585,0.002650695,0.0018137891,0.0020279833,0.0023673968,0.0016452036,0.0018506112,0.0021778245,0.0015238288,0.0017204144,0.0020450433,0.0014345231,0.0016259744,0.0019482705,0.0013731329,0.001561975,0.0018802665,0.001329639,0.0015156246,0.0018319507,0.0012991042,0.0014813977,0.0017928901,0.001279939,0.0014598139,0.0017673312,10.7426405,0.3634739,1342.3901,2.4910772

In [81]:
sep_max = 1.45 * u.deg
sep_min = 0.15 * u.deg
SGA_match = []
with open("rosette_match_output_dist.txt", "a") as f:
    for i in range(len(centers)):
        print('TILE ID: ', centers[i]['TILEID'], file = f)
        ra, dec = centers[i]['RA'] * u.deg, centers[i]['DEC'] * u.deg
        center = SkyCoord(ra, dec)
        for j in range(len(disttab)):
            ra2, dec2 = disttab[j]['RA'] * u.deg, disttab[j]['DEC'] * u.deg
            target = SkyCoord(ra2, dec2)
            d2d = center.separation(target)
            msk1 = d2d < sep_max
            msk2 = d2d > sep_min
            if msk1 == True and msk2 == True:
                # print('Match found')
                SGA_match.append(disttab[j]['SGA_ID'])
                print('Match found:', disttab[j]['SGA_ID'], file = f)
            # print()
            # break
        print(file = f)
        # break
# f.close()

In [80]:
f.close()

In [84]:
(np.unique(SGA_match))

array([  49524,  135945,  294387,  464075,  474506,  497874,  630029,
        662902,  772284, 1065602, 1271764, 1351452, 1385252, 1413430,
       2000090])

In [85]:
disttab[disttab['SGA_ID'] == 2000090]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog,DM1_SN,e_DM1_SN,DM2_SN,e_DM2_SN,DM3_SN,e_DM3_SN,DM4_SN,e_DM4_SN,DM5_SN,e_DM5_SN,DM6_SN,e_DM6_SN,DM7_SN,e_DM7_SN,SN_Catalog
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float64,float64,float64,str16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,str24
2000090,SGA-2020 2000090,KKH86,2807150,208.639395,4.244438,dIrr,177.0,1.12,0.61,0.00096053514,25.974571,17.1,False,LGDWARFS,528594,KKH86,1,True,208.639395,4.244438,1.12,2086p042,208.63972178060317,4.244498901726164,1.0676076,SB26,170.32007,0.5827334,208.63968031066798,4.24463009280064,22.80629,22.110266,23.175814,22.7685,-1.0,-1.0,-1.0,2.1870558,7.5076385,14.265005,21.140234,26.860487,32.02823,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,21.529501,21.079931,20.850012,19.002533,18.631569,18.403715,17.85994,17.516594,17.313553,17.326685,16.972342,16.754358,17.085537,16.721113,16.503862,16.940128,16.571085,16.35761,-1.0,-1.0,-1.0,0.05646121,0.083752856,0.14392973,0.12178976,0.14702602,0.2187881,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.032627493,0.03765607,0.06573504,0.010871151,0.012353091,0.018345706,0.00635901,0.0073078596,0.011273069,0.005092679,0.0059680473,0.008832604,0.004640285,0.005482721,0.008107432,0.0044362904,0.0052330135,0.007900765,16.524176,1.0284809,5.574703,2.0700536,0.8340654,16.116747,0.9855422,6.54034,2.072152,0.18118383,15.921342,0.9220686,7.505497,2.1670828,1.8919169,0,27.06,0.16,0.16,McConnachi,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0

In [88]:
# Change use_cached = False if we want to redo the query at some point.
use_cached = True

if os.path.exists('cache/tf_targets_everest.fits') and use_cached:
    tf_targets = Table.read('cache/tf_targets_everest.fits')
else:
    try:
        db = psycopg2.connect(host='decatdb.lbl.gov', database='desi', user='desi')
        cursor = db.cursor()
        
        query = """SELECT DISTINCT pr.pvtype, pr.sga_id, fd.targetid, fd.fiber_ra, fd.fiber_dec, fd.fiberstatus, fd.tileid, zd.target_ra, zd.target_dec, zd.z, zd.zerr, zd.spectype, zd.deltachi2, zd.zwarn
                   FROM everest.ztile_cumulative_fibermap fd
                   INNER JOIN
                       everest.ztile_cumulative_redshifts zd ON zd.targetid = fd.targetid
                   INNER JOIN pv.proposals pr ON (CAST(pr.ra*10000 AS INTEGER) = CAST(zd.target_ra*10000 AS INTEGER)) AND 
                                                 (CAST(pr.dec*10000 AS INTEGER) = CAST(zd.target_dec*10000 AS INTEGER))
                   WHERE (pr.pvtype LIKE 'TFT' or pr.pvtype LIKE 'EXT' or pr.pvtype LIKE 'SGA');"""
        
        cursor.execute(query)
        rows = cursor.fetchall()
        tf_targets = Table(list(map(list, zip(*rows))), 
                           names=['PVTYPE', 'SGA_ID', 'TARGETID', 'FIBER_RA', 'FIBER_DEC', 'FIBERSTATUS', 'TILEID', 
                                  'TARGET_RA', 'TARGET_DEC', 'Z', 'ZERR', 'SPECTYPE', 'DELTACHI2', 'ZWARN'])
        tf_targets.write('cache/tf_targets_everest.fits', overwrite=True)
        
    except (Exception, psycopg2.errors) as error:
        print(error)
    finally:
        if db is not None:
            db.close()
            
tf_targets

PVTYPE,SGA_ID,TARGETID,FIBER_RA,FIBER_DEC,FIBERSTATUS,TILEID,TARGET_RA,TARGET_DEC,Z,ZERR,SPECTYPE,DELTACHI2,ZWARN
bytes3,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,bytes6,float64,int64
EXT,17643,39633062056692288,0.0,0.0,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0
EXT,17643,39633062056692288,115.163671928495,39.2330481232594,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0
EXT,17643,39633062056692288,115.163676763292,39.2330716810911,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0
EXT,17643,39633062056692288,115.163687172584,39.2330595735952,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0
EXT,17643,39633062056692288,115.163687272099,39.2330517354585,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0
EXT,17643,39633062056692288,115.163687614895,39.2330239448729,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0
EXT,17643,39633062056692288,115.163692746003,39.2330238257014,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0
EXT,17643,39633062056692288,115.163702566258,39.2330592761066,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0
EXT,17643,39633062056692288,115.163702811737,39.2330394673576,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0


In [89]:
l = np.unique(SGA_match)
match = []
SGA_ids = list(tf_targets['SGA_ID'])
for l_id in l:
    for SGA_id in SGA_ids:
        if SGA_id == l_id:
            # print(SGA_id)
            match.append(SGA_id)

In [91]:
len(np.unique(match))

12

## Testing Multiple values of Distance Moduli

In [23]:
S_dist = sntab['S_mu0']
S_err = sntab['S_emu0']
S2_dist = sntab['S2_mu0']
S2_err = sntab['S2_emu0']
M31_dist = sntab['M31_mu0']
M31_err = sntab['M31_emu0']
M17_dist = sntab['M17_mu0']
M17_err = sntab['M17_emu0']

NameError: name 'sntab' is not defined

In [ ]:
plt.figure(figsize = (13, 4))
plt.errorbar(range(len(sntab)), S_dist, fmt = '.', yerr = S_err, label = 'SALT', alpha = 0.2)
plt.errorbar(range(len(sntab)), S2_dist, fmt = '.', yerr = S2_err, label = 'SALT2', alpha = 0.2)
plt.errorbar(range(len(sntab)), M31_dist, fmt = '.', yerr = M31_err, label = 'M31', alpha = 0.2)
plt.errorbar(range(len(sntab)), M17_dist, fmt = '.', yerr = M17_err, label = 'M17', alpha = 0.2)
plt.legend()

In [ ]:
plt.errorbar(S_dist, S2_dist, fmt = '.', xerr = S_err, yerr = S2_err, alpha = 0.2)
plt.xlabel('SALT')
plt.ylabel('SALT2')
plt.plot([34.5, 38.5], [34.5, 38.5], 'k:');

In [ ]:
plt.errorbar(S_dist, M31_dist, fmt = '.', xerr = S_err, yerr = M31_err, alpha = 0.2)
plt.xlabel('SALT')
plt.ylabel('M31')
plt.plot([34.5, 38.5], [34.5, 38.5], 'k:');

In [ ]:
plt.errorbar(S2_dist, M31_dist, fmt = '.', xerr = S2_err, yerr = M31_err, alpha = 0.2)
plt.xlabel('SALT2')
plt.ylabel('M31')
plt.plot([33, 38.5], [33, 38.5], 'k:');

In [ ]:
plt.errorbar(M17_dist, M31_dist, fmt = '.', xerr = M17_err, yerr = M31_err, alpha = 0.2)
plt.xlabel('M17')
plt.ylabel('M31')
plt.plot([33, 38.5], [33, 38.5], 'k:');